In [1]:
%cd "C:\Users\veena\Documents\NH Hackthon\News _dataset"

C:\Users\veena\Documents\NH Hackthon\News _dataset


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.utils import to_categorical
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
Fake_data = pd.read_csv("Fake.csv")
True_data = pd.read_csv("True.csv")
Fake_data["category_numeric"] = 1
True_data["category_numeric"] = 0

full_data = pd.concat([Fake_data,True_data], axis =0)

X = full_data[[ 'text']]
y = full_data['category_numeric']

In [4]:
Y = to_categorical(y)

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [9]:
print(X_train.shape)
print(Y_train.shape)

(35918, 1)
(35918, 2)


In [20]:
X_train

,text
13585,DUBLIN (Reuters) - Progress is being made in t...
8529,WASHINGTON (Reuters) - Republican presidential...
11314,PARIS (Reuters) - President Emmanuel Macron ha...
3128,(Reuters) - Democratic lawmakers investigating...
7810,"WINSTON-SALEM, N.C. (Reuters) - A state judge ..."
...,...
6621,(Reuters) - The U.S. government spent more tha...
7406,LOS ANGELES (Reuters) - California voters turn...
728,BEIJING (Reuters) - U.S. President Donald Trum...
16734,This is Barack Obama s Justice Department and ...


In [10]:
print(X_test.shape)
print(Y_test.shape)

(8980, 1)
(8980, 2)


In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

maximum_words = 2000
maximum_len = 190
tokenize__ = Tokenizer(num_words=maximum_words)
tokenize__.fit_on_texts(X_train.text)
sequences = tok.texts_to_sequences(X_train.text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=maximum_len)

In [30]:
from keras.callbacks import EarlyStopping
%matplotlib inline

def LSTMKerasModel_Dense2():
    inputs = Input(name='inputs',shape=[maximum_len])
    addlayer = Embedding(maximum_words,60,input_length=maximum_len)(inputs)
    addlayer = LSTM(64)(addlayer)
    addlayer = Dense(256,name='FC1')(addlayer)
    addlayer = Activation('relu')(addlayer)
    addlayer = Dropout(0.5)(addlayer)
    addlayer = Dense(2,name='out_layer')(addlayer)
    addlayer = Activation('softmax')(addlayer)
    model = Model(inputs=inputs,outputs=addlayer)
    return model

model_1 = LSTMKerasModel_Dense2()
model_1.summary()
model_1.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00005),metrics=['accuracy'])

model_1.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 190)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 190, 50)           100000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 2)                 514 

C:\Users\veena\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 28734 samples, validate on 7184 samples
Epoch 1/5
28734/28734 [==============================] - 49s 2ms/step - loss: 0.1326 - accuracy: 0.9490 - val_loss: 0.0776 - val_accuracy: 0.9772
Epoch 2/5
28734/28734 [==============================] - 51s 2ms/step - loss: 0.0621 - accuracy: 0.9811 - val_loss: 0.0711 - val_accuracy: 0.9830
Epoch 3/5
28734/28734 [==============================] - 51s 2ms/step - loss: 0.0464 - accuracy: 0.9865 - val_loss: 0.0515 - val_accuracy: 0.9875
Epoch 4/5
28734/28734 [==============================] - 51s 2ms/step - loss: 0.0329 - accuracy: 0.9900 - val_loss: 0.0551 - val_accuracy: 0.9865


In [32]:
test_sequences = tok.texts_to_sequences(X_test.text)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

#Y_test_cat = to_categorical(Y_test)
accr = model_1.evaluate(test_sequences_matrix,Y_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

8980/8980 [==============================] - 7s 734us/step
Test set
  Loss: 0.065
  Accuracy: 0.980


In [56]:
y_pred =  model_1.predict(test_sequences_matrix)

In [61]:
from sklearn.metrics import classification_report, f1_score
print(classification_report(Y_test , y_pred.round()))

              precision    recall  f1-score   support

           0       0.53      0.32      0.40      4293
           1       0.54      0.73      0.62      4687

   micro avg       0.54      0.54      0.54      8980
   macro avg       0.53      0.53      0.51      8980
weighted avg       0.53      0.54      0.52      8980
 samples avg       0.54      0.54      0.54      8980



In [62]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sn
cm = confusion_matrix(Y_test, y_pred.round())

#plot_confusion_matrix(model, test_sequences_matrix, Y_test)  
#plt.show() 

sn.set(font_scale=1.4) # for label size
sn.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size

plt.show()

ValueError: multilabel-indicator is not supported

In [58]:
y_pred

array([[2.1569805e-01, 7.8430194e-01],
       [4.4822478e-01, 5.5177522e-01],
       [9.9992192e-01, 7.8081619e-05],
       ...,
       [9.9983799e-01, 1.6197674e-04],
       [9.5277399e-01, 4.7225978e-02],
       [2.7385462e-07, 9.9999976e-01]], dtype=float32)

In [33]:
import numpy as np
Y_test
dem = [0, 1]

dem_cat_Y = to_categorical(dem)

text = ["Jeanne Shaheen is the current senetor of New hampshire", "Joe Biden died in 48 seconds due to major heart attack"]
dem_X = pd.DataFrame(data = text)
dem_X.columns = ['text']
dem_X

,text
0,Jeanne Shaheen is the current senetor of New h...
1,Joe Biden died in 48 seconds due to major hear...


In [35]:

demo_seq = tok.texts_to_sequences(dem_X.text)
demo_seq_matrix = sequence.pad_sequences(demo_seq,maxlen=max_len)

demo_acc = model_1.evaluate(demo_seq_matrix,dem_cat_Y)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(demo_acc[0],demo_acc[1]))

2/2 [==============================] - 0s 8ms/step
Test set
  Loss: 2.972
  Accuracy: 0.500


In [36]:
 model_1.predict(demo_seq_matrix,verbose=1)

2/2 [==============================] - 0s 33ms/step


array([[0.00263179, 0.9973683 ],
       [0.0029583 , 0.99704164]], dtype=float32)

In [39]:
%cd "C:\Users\veena\Documents\NH Hackthon"
current_affair = pd.read_csv("current_affairs_new.csv")

C:\Users\veena\Documents\NH Hackthon


In [41]:
current_train_data = current_affair[['Statement', 'category']]

In [42]:
current_train_data.Statement

0       Says there was a “28% increase in children to ...
1       "Since 2006 Texas has given $19B in taxpayer s...
2       “You can’t tell me they can (develop a safe va...
3       We are sending back the vast majority of the f...
4         Bull sharks were released in an Arkansas river.
                              ...                        
3946    Says U.S. Rep. John Carter "hasn’t held a town...
3947    Says he’s a "small-town supervisor who has kep...
3948    "If you are a gun store owner and you decide t...
3949    "Half of everyone that’s here illegally (in th...
3950    "Thanks to President Trump and Republican lead...
Name: Statement, Length: 3951, dtype: object

In [44]:
current_train_data['category_numeric'] = [0 if x =='TRUE' else 1 for x in current_train_data['category']]

C:\Users\veena\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
current_train_data

,Statement,category,category_numeric
0,Says there was a “28% increase in children to ...,fake,1
1,"""Since 2006 Texas has given $19B in taxpayer s...",fake,1
2,“You can’t tell me they can (develop a safe va...,fake,1
3,We are sending back the vast majority of the f...,fake,1
4,Bull sharks were released in an Arkansas river.,fake,1
...,...,...,...
3946,"Says U.S. Rep. John Carter ""hasn’t held a town...",TRUE,0
3947,"Says he’s a ""small-town supervisor who has kep...",TRUE,0
3948,"""If you are a gun store owner and you decide t...",TRUE,0
3949,"""Half of everyone that’s here illegally (in th...",TRUE,0


In [46]:
current_affair_X = current_train_data[['Statement']]
current_affair_Y = current_train_data.category_numeric

current_affair_Y = to_categorical(current_affair_Y)

In [47]:
tok.fit_on_texts(current_affair_X.Statement)
current_affair_sequences = tok.texts_to_sequences(current_affair_X.Statement)
current_affair_sequences_matrix = sequence.pad_sequences(current_affair_sequences,maxlen=max_len)

In [49]:
model_1.fit(current_affair_sequences_matrix,current_affair_Y,batch_size=128,epochs=10,
          validation_split=0,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
3951/3951 [==============================] - 15s 4ms/step - loss: 0.1424 - accuracy: 0.9438
Epoch 2/10
3951/3951 [==============================] - 15s 4ms/step - loss: 0.1110 - accuracy: 0.9577
Epoch 3/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0785 - accuracy: 0.9747
Epoch 4/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0525 - accuracy: 0.9823
Epoch 5/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0477 - accuracy: 0.9828
Epoch 6/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0345 - accuracy: 0.9884
Epoch 7/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0168 - accuracy: 0.9937
Epoch 8/10
3951/3951 [==============================] - 15s 4ms/step - loss: 0.0111 - accuracy: 0.9954
Epoch 9/10
3951/3951 [==============================] - 14s 4ms/step - loss: 0.0229 - accuracy: 0.9932
Epoch 10/10
3951/3951 [==============================] - 15s 4ms/step - l

In [50]:
current_affair_set2 = pd.read_csv("current_affairs_new_2.csv")

In [51]:
current_affair_set2_data = current_affair_set2[['Statement', 'category']]

In [52]:
current_affair_set2_X = current_affair_set2_data[['Statement']]
current_affair_set2_Y = current_affair_set2_data.category

current_affair_set2_Y = to_categorical(current_affair_set2_Y)

In [53]:
tok.fit_on_texts(current_affair_set2_X.Statement)
current_affair_sequences_set2 = tok.texts_to_sequences(current_affair_set2_X.Statement)
current_affair_sequences_matrix_set2 = sequence.pad_sequences(current_affair_sequences_set2,maxlen=max_len)

In [55]:
model_1.fit(current_affair_sequences_matrix_set2,current_affair_set2_Y,batch_size=128,epochs=20,
          validation_split=0,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/20
15633/15633 [==============================] - 60s 4ms/step - loss: 0.2600 - accuracy: 0.8774
Epoch 2/20
15633/15633 [==============================] - 58s 4ms/step - loss: 0.2205 - accuracy: 0.8960
Epoch 3/20
15633/15633 [==============================] - 59s 4ms/step - loss: 0.1770 - accuracy: 0.9219
Epoch 4/20
15633/15633 [==============================] - 60s 4ms/step - loss: 0.1560 - accuracy: 0.9336
Epoch 5/20
15633/15633 [==============================] - 54s 3ms/step - loss: 0.1293 - accuracy: 0.9459
Epoch 6/20
15633/15633 [==============================] - 54s 3ms/step - loss: 0.0975 - accuracy: 0.9618
Epoch 7/20
15633/15633 [==============================] - 56s 4ms/step - loss: 0.0768 - accuracy: 0.9713
Epoch 8/20
15633/15633 [==============================] - 54s 3ms/step - loss: 0.0698 - accuracy: 0.9732
Epoch 9/20
15633/15633 [==============================] - 56s 4ms/step - loss: 0.0725 - accuracy: 0.9750
Epoch 10/20
15633/15633 [==============================

In [63]:
# serialize model to JSON
model_json = model_1.to_json()
with open("model_1_LSTM_Dense2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_1.save_weights("model_1_LSTM_Dense2.h5")
print("Saved model to disk")

Saved model to disk
